In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier,Pool
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.svm import SVC


import numpy as np

# get the absolute path of the running python file
code_filepath = os.path.abspath("__file__")

titanic_dir = os.path.dirname(os.path.dirname(code_filepath))

# get the data file
data_filepath = os.path.join(titanic_dir, "data", "train.csv")

# read .csv data file to pandas data frame
train_df = pd.read_csv(data_filepath)


test_path= os.path.join(titanic_dir, "data", "test.csv")
test_df= pd.read_csv(test_path)

FileNotFoundError: [Errno 2] File C:\Users\Dat Nguyen\Kaggle\data\train.csv does not exist: 'C:\\Users\\Dat Nguyen\\Kaggle\\data\\train.csv'

In [ ]:
train_df.head()

In [ ]:
#checking for null 
train_df.info()


In [ ]:
# Fill na values in Cabin with Unknown:
for d in [train_df, test_df]:
    d['Cabin'].fillna('Unknown', inplace=True)


In [ ]:
#replace missing age with passengers average age
data= pd.concat([train_df,test_df], sort=False)
for d in [train_df, test_df]:
    d['Age'].fillna(data['Age'].mean(), inplace=True)

In [ ]:
#Fill na values in Embarked columns:
for d in [train_df, test_df]:
    d['Embarked'].fillna(data['Embarked'].mode()[0], inplace= True)

In [ ]:
#Create family size features by combining SibSp and Parch:
for d in [train_df, test_df]:
    d['Family_size']= d['SibSp']+ d['Parch']+1

In [ ]:
#extracting Initial from Names: 
for d in [train_df, test_df]:
    d['Title'] = d['Name'].str.extract('([A-Za-z]+)\.', expand=False)
    d['Title'] = d['Title'].replace('Ms', 'Miss')
    d['Title'] = d['Title'].replace('Mlle', 'Miss')
    d['Title'] = d['Title'].replace('Mme', 'Mrs')
    d['Title'] = d['Title'].apply(lambda x: 'Other' if ((x != 'Mr') and (x != 'Miss') and (x != 'Mrs')) else x)#
print(train_df['Title'].value_counts().sort_values())
print(test_df['Title'].value_counts().sort_values())


In [ ]:
#Extracting Decks from Cabin columns
for d in [train_df, test_df]:
    d['Decks']= d['Cabin'].str.extract('([A-Z])', expand=False)
    print(d['Decks'].value_counts())
train_df[train_df['Decks']=='T']

In [ ]:
for d in [train_df, test_df]:

    for index, value in d['Decks'].iteritems():
        if value in ['A','B','C']:
            d.loc[index,'Deck_level']= 'Top'
        elif value in ['D','E']:
            d.loc[index,'Deck_level']= 'Upper'
        elif value == 'F':
            d.loc[index,'Deck_level']= 'Middle'
        elif value == 'U':
            d.loc[index,'Deck_level']= 'Unknown'
        else: 
            d.loc[index,'Deck_level']= 'Low'
print(train_df['Deck_level'].value_counts())
print(test_df['Deck_level'].value_counts())

In [ ]:
for d in [train_df, test_df]:

    for index, value in d['Age'].iteritems():
        if value<=15:
            d.loc[index,'Age_group']= 'Children'
        elif 15< value <= 50:
            d.loc[index,'Age_group']= 'Adults'
        else: 
            d.loc[index,'Age_group']= 'Elder'

In [ ]:
test_df['Fare'].fillna(test_df['Fare'].mean(), inplace=True)

In [ ]:
train_df.describe(include='all')

# Stacking 5 models:
## 2 Trees: Catboost, LightGBM
## 2 Non-trees: KNN, Logistic Regression
## 1 NNs: MLPClassifier
## Metamodel: Logistic Regression


Catboost & Light GBM


In [ ]:
tree_train= train_df.drop(['Survived','Ticket','PassengerId','Name','SibSp','Parch','Decks'], axis=1)
tree_label= train_df['Survived']
tree_test= test_df.drop(['Ticket','PassengerId','Name','SibSp','Parch','Decks'], axis=1)
catdims= ['Pclass','Sex', 'Cabin', 'Embarked', 'Title', 'Deck_level', 'Age_group']

for d in [tree_train,tree_test]:
    for col in catdims:
        d[col]=d[col].astype('category')
    

print(tree_train.head())
print(tree_test.head())

KNN, Logitistic Regresssions & NN

In [ ]:
#try again with less features:
ntree_train = train_df.drop(['Survived','Name','PassengerId','SibSp','Parch','Cabin','Ticket','Decks','Title'], axis= 1)
ntree_test = test_df.drop(['Name','PassengerId','SibSp','Parch','Cabin','Ticket','Decks','Title'], axis= 1)
ntree_label= train_df['Survived']
ntree_train= pd.get_dummies(ntree_train)
ntree_test= pd.get_dummies(ntree_test)

scaler= preprocessing.StandardScaler()

train_scaled= scaler.fit_transform(ntree_train)

test_scaled= scaler.fit_transform(ntree_test)
ntree_train_scaled= pd.DataFrame(train_scaled, columns= ntree_train.columns)
ntree_test_scaled= pd.DataFrame(test_scaled, columns=ntree_test.columns)

ntree_train.info()

In [ ]:
def features_stacking(X, Y, X_test, model, cv = 4, name_model_as_feature = 'logistic', 
                      catdims= None, catboost=False, lightgbm=False, random_state=False):
    skf = StratifiedKFold(n_splits=cv, shuffle = True)
    accuracy_list=[]
    # Create an empty dictionary to keep k-fold
    train_splits = {}
    # Loop for k-fold index to generate X_train, X_val, Y_train, Y_val of each fold
    for (i, (train_index, val_index)) in enumerate(skf.split(X, Y)):
        train_splits['Fold_' + str(i + 1)] = (train_index, val_index, X.iloc[train_index], X.iloc[val_index], 
                                              Y.iloc[train_index], Y.iloc[val_index])
    # Create an empty DataFrame to keep features stacking of training data
    stacking_train = pd.DataFrame(columns = [name_model_as_feature])
    # Create an empty array to compute sum of probability of test data
    sum_proba_test = np.zeros(X_test.shape[0])
    # Loop for k_fold
    for i in range(cv):
        # Retrieve infomation of fold_i+1
        (train_index, val_index, X_train, X_val, Y_train, Y_val) = train_splits['Fold_' + str(i + 1)]
        # Set random_state for the model and fit
        if random_state:    
            model.set_params(random_state = np.random.randint(2019))
        if catboost:
            model.set_params(cat_features=catdims)
        if lightgbm:
            model.set_params(categorical_feature=catdims)
        model.fit(X_train, Y_train)
        # Append results on X_val to stacking_train DataFrame
        stacking_train = pd.concat([stacking_train, pd.DataFrame(model.predict_proba(X_val)[:,1], columns = [name_model_as_feature], index = val_index)], axis = 0)
        # Compute predict_proba of X_test
        sum_proba_test += model.predict_proba(X_test)[:,1]
        y_pred= model.predict(X_val)
        accuracy = metrics.accuracy_score(Y_val, y_pred)
        accuracy_list.append(accuracy)
        accuracy= np.mean(accuracy_list)
    print('Model accuracy:', accuracy)    
    
    # Reset index of stacking_train
    stacking_train = stacking_train.sort_index()
        
    # Create a DataFrame to keep stacking_test
    stacking_test = pd.DataFrame(data = sum_proba_test/cv, columns = [name_model_as_feature])
    return (stacking_train, stacking_test)
  
  
    


In [ ]:
# Logistic Regression
model = LogisticRegression(C=100)
logistic = features_stacking(ntree_train_scaled, ntree_label, ntree_test_scaled, model, cv = 4, name_model_as_feature = 'logistic')


In [ ]:
# KNN
model = KNeighborsClassifier()
knn = features_stacking(ntree_train_scaled, ntree_label, ntree_test_scaled,model, cv = 4, name_model_as_feature = 'knn', random_state = False)


In [ ]:
# SVM
model = SVC(probability = True, gamma='scale')
svm = features_stacking(ntree_train_scaled, ntree_label, ntree_test_scaled, model, cv = 4, name_model_as_feature = 'svm', random_state=2019)

In [ ]:
model = MLPClassifier()
nn = features_stacking(ntree_train_scaled, ntree_label, ntree_test_scaled, model, cv = 4, name_model_as_feature = 'nn', random_state=2019)

In [ ]:
cb_catdims=[0,1,4,5,7,8,9]
model = CatBoostClassifier()
cb = features_stacking(tree_train, tree_label, tree_test, 
                       model, cv = 4, name_model_as_feature = 'cb', random_state = False,catboost=True, catdims=cb_catdims)

In [ ]:
#LightGBM
model = LGBMClassifier(learning_rate=0.01, n_estimators=1000, num_leaves=10)
gbm = features_stacking(tree_train, tree_label, tree_test, 
                       model, cv = 4, name_model_as_feature = 'gbm', random_state = 2019,lightgbm=True)

In [ ]:
stacking_train = pd.concat([knn[0], svm[0], nn[0], cb[0], gbm[0]], axis = 1)
stacking_train

In [ ]:
stacking_test = pd.concat( [knn[1], svm[1], nn[1], cb[1], gbm[1]], axis = 1)

In [ ]:
sns.heatmap(stacking_train.corr())

In [ ]:
clf = LogisticRegression()

k_fold = StratifiedKFold(n_splits=10, shuffle= True,random_state=1906)
params = {'penalty': ['l1','l2'], 
                  'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000]}

gs_clf = GridSearchCV(clf , param_grid = params, cv=k_fold, scoring="accuracy", verbose = 1, n_jobs=-1)
gs_clf.fit(stacking_train, train_df['Survived'])

clf_best = gs_clf.best_estimator_
# Best score
gs_clf.best_score_  

In [ ]:
clf_best

In [ ]:
Y_predict = clf_best.predict(stacking_test)
submission = pd.DataFrame({'PassengerId': test_df['PassengerId'], 'Survived': Y_predict.astype(int)})
submission.to_csv('stacking.csv', index = False)